In [1]:
from config.config import BOOK_API_KEY
from googleapiclient.discovery import build
from IPython.display import JSON
import pandas as pd

In [2]:
api_service_name = "books"
api_version = "v1"

book_ids = [
    "JfLsAQAAQBAJ",  # Replace with actual book IDs
    # Can add more book IDs
]

# Get credentials and create an API client

books = build(api_service_name, api_version, developerKey=BOOK_API_KEY)

request = books.volumes().list(
    q=",".join(book_ids)
)
response = request.execute()

print(response)


{'kind': 'books#volumes', 'totalItems': 1, 'items': [{'kind': 'books#volume', 'id': 'JfLsAQAAQBAJ', 'etag': '6OvUDJ6f1Lo', 'selfLink': 'https://books.googleapis.com/books/v1/volumes/JfLsAQAAQBAJ', 'volumeInfo': {'title': 'Hamlet', 'authors': ['Shakespeare, William'], 'publisher': 'Aegitas', 'publishedDate': '2015-04-24', 'description': 'Hamlet é uma tragédia de William Shakespeare, escrita entre 1599 e 1601. A peça, situada na Dinamarca, reconta a história de como o Príncipe Hamlet tenta vingar a morte de seu pai Hamlet, o rei, executando seu tio Cláudio, que o envenenou e em seguida tomou o trono casando-se com a mãe de Hamlet. A peça traça um mapa do curso de vida na loucura real e na loucura fingida — do sofrimento opressivo à raiva fervorosa — e explora temas como a traição, vingança, incesto, corrupção e moralidade. Apesar da enorme investigação que se faz acerca do texto, o ano exato em que Shakespeare escreveu-o permanece em debate. Três primeiras versões da peça sobrevivem aos 

In [3]:
JSON(response)

<IPython.core.display.JSON object>

In [4]:
api_service_name = "books"
api_version = "v1"

book_ids = [
    "JfLsAQAAQBAJ",
    "ksOEEAAAQBAJ",
    "LPKyUG4OmlsC",
    "vAsdtG_P21oC"
]

# Get credentials and create an API client

books = build(api_service_name, api_version, developerKey=BOOK_API_KEY)

In [5]:
def get_book_info(books, book_ids):
    all_data = []

    """request = books.volumes().list(
        q=",".join(book_ids)
    )

    response = request.execute()

    # loop through items in response
    for item in response.get("items", []):
        volume_info = item.get("volumeInfo", {})"""

    for book_id in book_ids:
        request = books.volumes().get(volumeId=book_id)
        response = request.execute()

        volume_info = response.get("volumeInfo", {})

        data = {
            'title': volume_info.get('title', 'N/A'),
            'authors': volume_info.get('authors', 'N/A'),
            'publishedDate': volume_info.get('publishedDate', 'N/A'),
            'description': volume_info.get('description', 'N/A'),
            'pageCount': volume_info.get('pageCount', 'N/A'),
            'categories': volume_info.get('categories', 'N/A'),
        }
        all_data.append(data)

    return pd.DataFrame(all_data)


In [6]:
book_data = get_book_info(books, book_ids)

In [7]:
book_data

,title,authors,publishedDate,description,pageCount,categories
0,Hamlet,"[Shakespeare, William]",2015-04-24,"Hamlet é uma tragédia de William Shakespeare, ...",142,"[Drama / General, Drama / Shakespeare, Drama /..."
1,Macbeth - NE,[William Shakespeare],2012-01-01,O apaixonante deste texto de Shakespeare é o c...,114,[Drama / General]
2,Odisseia,[Homero],2007-04-25,"Surgida no século IX a.C., a ""Odisseia"" é cons...",456,[Fiction / Classics]
3,Noite de Reis,[William Shakespeare],2011-11-09,"Nesta peça, um grupo arma uma cilada para faze...",144,[Fiction / General]


In [8]:
def get_popular_books_by_category(books, category):
    # Fonksiyon, belirli bir kategorideki popüler kitapları almak için kullanılır.
    all_data = []

    # Google Books API'ye bir sorgu yapılıyor.
    request = books.volumes().list(
        q=f"subject:{category}",  # Belirli bir kategoriye ait kitapları sorgulamak için kullanılan sorgu ifadesi.
        orderBy="relevance",  # Kitapların sıralama ölçütü. "relevance" en ilgili olanları ön plana çıkarır.
        maxResults=40  # En fazla alınacak kitap sayısı.
    )

    # API'ye yapılan sorgu sonucunda gelen yanıt alınıyor.
    response = request.execute()

    # Gelen yanıt içindeki her bir kitap için bir döngü oluşturuluyor.
    for item in response.get("items", []):
        # Kitap bilgilerini içeren kısmı alıyoruz.
        volume_info = item.get("volumeInfo", {})

        # Kitap bilgileri bir sözlük içinde düzenleniyor.
        data = {
            'title': volume_info.get('title', 'N/A'),
            'authors': volume_info.get('authors', ['N/A']),
            'publishedDate': volume_info.get('publishedDate', 'N/A'),
            'description': volume_info.get('description', 'N/A'),
            'pageCount': volume_info.get('pageCount', 'N/A'),
            'categories': volume_info.get('categories', ['N/A']),
        }
        
        # Kitap bilgileri listeye ekleniyor.
        all_data.append(data)

    # Oluşturulan kitap bilgileri listesi bir Pandas DataFrame'e dönüştürülüyor ve geri döndürülüyor.
    return pd.DataFrame(all_data)

##### Aşağıda categories kısmında "Literary Criticism" gibi Drama'dan farklı kategoriler gözükmektedir bunun sebebi Google Books API, bir kitabın kategorilerini belirlemek için çeşitli kaynakları kullanır ancak kitaplar genellikle birkaç farklı kategoriyle etiketlenebilir ve bu etiketler kitabın içeriğine, konusuna veya bir dizi diğer faktöre dayanabilir bu nedenle API tarafından döndürülen bir kitabın kategorileri sadece belirli bir anahtar kelimeyle eşleşmeyebilir.

In [9]:
popular = get_popular_books_by_category(books, category="Drama")
popular

,title,authors,publishedDate,description,pageCount,categories
0,The Taming of the Shrew,[William Shakespeare],2002,The story of Katherina Minola and her marriage...,300,[Drama]
1,A Doll's House,[Henrik Ibsen],2008-08-19,A revised Methuen Student Edition of the class...,164,[Drama]
2,Othello,[William Shakespeare],2022-12-14,"""Othello"" – William Shakespeare (käännös Paavo...",147,[Drama]
3,Rosmersholm,[Henrik Ibsen],2022-10-07,Johannes Rosmer ist ein ehemaliger Pfarrer und...,101,[Fiction]
4,Othello,[William Shakespeare],2009-01-01,"Believed to have been written in 1603, Shakesp...",253,[Drama]
5,"The Iliad (World Classics, Unabridged)",[Homer],2016-09-01,The story of the Iliad centers on the critical...,406,[Drama]
6,The Storm,[Aleksandr Nikolaevich Ostrovsky],2022-09-16,DigiCat Publishing presents to you this specia...,91,[Drama]
7,Who's Afraid of Virginia Woolf?,[Edward Albee],2003-07-29,"George, a disillusioned academic, and Martha, ...",258,[Drama]
8,"Don Quixote (World Classics, Unabridged)",[Miguel de Cervantes Saavedra],2016-10-01,Don Quixote is a Spanish novel by Miguel de Ce...,1160,[Drama]
9,Bakkhai,[Euripides],2001-02-22,"Regarded by many as Euripides' masterpiece, Ba...",160,[Drama]


In [10]:
def get_book_info_and_url(books, query):
    request = books.volumes().list(q=query)
    response = request.execute()

    if response.get("items"):
        book_info = response["items"][0]["volumeInfo"]
        title = book_info.get("title", "")
        authors = ", ".join(book_info.get("authors", []))

        # Oluşturulan URL
        google_books_url = f"https://books.google.com.tr/books?id={response['items'][0]['id']}"

        return {
            'title': title,
            'authors': authors,
            'google_books_url': google_books_url
        }
    else:
        return None

In [11]:
info= get_book_info_and_url(books, query="Harry Potter")
info

{'title': 'Harry Potter ve Sirlar Odasi',
 'authors': 'J.K. Rowling',
 'google_books_url': 'https://books.google.com.tr/books?id=Dg5MEAAAQBAJ'}

In [12]:
def get_popular_books_by_category(books, category, max_results=10, total_results=50):
    all_data = []

    # İlk sorgu oluşturuluyor.
    request = books.volumes().list(
        q=f"subject:{category}",  # Belirli bir kategoriye ait kitapları sorgulamak için kullanılan sorgu ifadesi.
        orderBy="relevance",  # Kitapların sıralama ölçütü. "relevance" en ilgili olanları ön plana çıkarır.
        maxResults=max_results  # Her bir sorguda alınacak maksimum kitap sayısı.
    )

    # İlk sorgu gerçekleştiriliyor ve gelen yanıt alınıyor.
    response = request.execute()

    # Yanıt içindeki her bir kitap için bir döngü oluşturuluyor.
    for item in response.get("items", []):
        volume_info = item.get("volumeInfo", {})
        all_data.append(volume_info)

    # Toplam belirtilen sonuca ulaşana kadar sayfalar arasında döngü oluşturuluyor.
    while "nextPageToken" in response and len(all_data) <= total_results:
        # Bir sonraki sayfa için yeni bir sorgu oluşturuluyor.
        request = books.volumes().list(
            q=f"subject:{category}",  # Belirli bir kategoriye ait kitapları sorgulamak için kullanılan sorgu ifadesi.
            orderBy="relevance",  # Kitapların sıralama ölçütü. "relevance" en ilgili olanları ön plana çıkarır.
            maxResults=max_results,  # Her bir sorguda alınacak maksimum kitap sayısı.
            pageToken=response["nextPageToken"]  # Bir sonraki sayfanın belirteci.
        )

        # Yeni sorgu gerçekleştiriliyor ve gelen yanıt alınıyor.
        response = request.execute()

        # Yanıt içindeki her bir kitap için bir döngü oluşturuluyor.
        for item in response.get("items", []):
            volume_info = item.get("volumeInfo", {})
            all_data.append(volume_info)

    # Oluşturulan tüm kitap verileri bir listeden Pandas DataFrame'e dönüştürülerek geri döndürülüyor.
    return all_data

In [13]:
popular_book = get_popular_books_by_category(books, category="drama", max_results=10, total_results=30)
popular_book

[{'title': 'The Taming of the Shrew',
  'authors': ['William Shakespeare'],
  'publisher': 'Cambridge University Press',
  'publishedDate': '2002',
  'description': "The story of Katherina Minola and her marriage to Petruchio has been popular in the theatre for four centuries. The Taming of the Shrew's ongoing appeal is easy to understand - it offers almost production-proof farce, plus the perennially popular joke of the battle of the sexes - but Katherina's story, and the joke of taming an unruly woman have become increasingly controversial. This edition of The Taming of the Shrew examines how theatre directors and performers have explored the complexities of Katherina's story and that of Christopher Sly, the poor man whose story frames that of Katherine. The edition surveys a wide variety of theatrical interpretations of the play in the English-speaking world, particularly in the UK, North America, and Australia and New Zealand. It informs readers about precise details of the stage a

In [14]:
len(popular_book) 

10